In [1]:
# Import PySpark
from pyspark.sql import SparkSession

#Create SparkSession
spark = SparkSession.builder.appName('SparkCleaning').getOrCreate()

In [6]:
spark

In [3]:
df_spark = spark.read.csv("../datasets/imdb/validation_hidden.csv", inferSchema=True, header=True)
df_spark.repartition(4)

DataFrame[_c0: int, tconst: string, primaryTitle: string, originalTitle: string, startYear: string, endYear: string, runtimeMinutes: string, numVotes: double]

In [4]:
df_spark.show()

+---+---------+--------------------+-------------------+---------+-------+--------------+--------+
|_c0|   tconst|        primaryTitle|      originalTitle|startYear|endYear|runtimeMinutes|numVotes|
+---+---------+--------------------+-------------------+---------+-------+--------------+--------+
|  0|tt0003740|             Cabiria|               null|     1914|     \N|           148|  3452.0|
|  1|tt0008663|     A Man There Was|        Terje Vigen|     1917|     \N|            65|  1882.0|
|  3|tt0010307|           J'accuse!|               null|     1919|     \N|           166|  1692.0|
| 18|tt0014429|        Safety Last!|       Safety Last!|     1923|     \N|            74| 19898.0|
| 27|tt0015175|Die Nibelungen: S...|               null|     1924|     \N|           143|  5676.0|
| 39|tt0016332|       Seven Chances|               null|     1925|     \N|            56|  9914.0|
| 61|tt0018737|       Pandora's Box|               null|       \N|   1929|           109| 10475.0|
| 65|tt001

In [5]:
df_spark.count()

955